# Import Libraries

In [1]:
import urllib.request, json 
import pandas as pd
import numpy as np
import requests
import pandas_profiling
import spacy
from html import unescape
import re
from collections import defaultdict
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
sns.set()
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
from sklearn.decomposition import TruncatedSVD, PCA, NMF
from sklearn.metrics.pairwise import cosine_similarity
import gensim

# Collection DATA with Pushshift API

In [1]:
import urllib.request, json 
import pandas as pd
import numpy as np
import requests

In [8]:
# #run a sample
# with urllib.request.urlopen("https://api.pushshift.io/reddit/comment/search/?subreddit=leagueoflegends&after=1546318800&before=1564631940&sort_type=score&sort=desc&size=1000&score>500&score<800") as url:
#     sample = pd.DataFrame(data = json.loads(url.read().decode())['data'])

In [4]:
def get_data(after, before):
    url = 'https://api.pushshift.io/reddit/comment/search/?size=1000&after='+str(after)+'&before='+str(before)+'&subreddit=leagueoflegends'
    with urllib.request.urlopen(url) as url:
        df = pd.DataFrame(data = json.loads(url.read().decode())['data'])[['created_utc','author','score','body','subreddit']]
    return df

Note:
Use to convert time: https://www.unixtimestamp.com/index.php

Jul
- After(7/1/2018) 1530403200
- Before(7/31/2018) 1533081599

Aug
- After(8/1/2018) 1533081600
- Before(8/31/2018) 1535759999

Sep
- After(9/1/2018) 1535760000
- Before(9/30/2018) 1538308799

Oct
- After(10/1/2018) 1538352000
- Before(10/31/2018) 1541030399

Nov
- After(11/1/2018) 1541030400
- Before(11/30/2018) 1543622399

Dec
- After(12/1/2018) 1543622400
- Before(12/31/2018) 1546300799

Jan
- After(1/1/2019) 1546300800
- Before(1/31/2019) 1548935999

Feb
- After(2/1/2019) 1548979200
- Before(2/28/2019) 1551398399

Mar
- After(3/1/2019) 1551398400
- Before(3/31/2019) 1554076799

Apr
- After(4/1/2019) 1554076800
- Before(4/30/2019) 1556668799

May
- After(5/1/2019) 1556668800
- Before(5/31/2019) 1559347199

June
- After(6/1/2019) 1559347200
- Before(6/31/2019) 1561939199


6/12/19 1560297600
6/18/19 1560902399



In [26]:
df = get_data(1543622400, 1546300799)
# Will run until all posts have been gathered 
# from the 'after' date up until before date
error502 = 0
while len(df) > 0:
    # Calls get_data() with the created date of the last submission
    try:
        after = df['created_utc'].tail(1).item()
        df = df.append(get_data(after, 1546300799), ignore_index=True)
    except requests.exceptions.HTTPError:
        error502 += 1
        print(error502)
        after = df['created_utc'].tail(1).item()
        df = df.append(get_data(after, 1546300799), ignore_index=True)
    except KeyError:
        print(error502)
        break
    if error502 > 30:
        print(error502 + ' too many try')
        break
        
print(len(df))

0
431783


In [28]:
df.shape

(431783, 5)

In [29]:
#df.to_pickle('cm_18_12')

# Data Cleaning

In [4]:
df = pd.read_pickle('cm_19_06')

In [5]:
#pf = pandas_profiling.ProfileReport(df=df)
#pf.to_file(outputfile="output.html")

In [6]:
df.head()

,created_utc,author,score,body,subreddit
0,1560297603,Maikiol,1,"This happened so often in LAS, but in comparis...",leagueoflegends
1,1560297606,supertbagman,1,o7,leagueoflegends
2,1560297609,GalaxyVox,59,That Alistar... It’s just too good,leagueoflegends
3,1560297610,tim466,5,"Riot is something called a ""company"". This has...",leagueoflegends
4,1560297613,narfidy,1,Caitlyn gets 2 AD while enchanters are on the ...,leagueoflegends


In [5]:
df.drop_duplicates(subset='body', keep=False, inplace=True)

In [6]:
df = df[df['author']!='Ilackfocus']
df = df[df['author']!='[deleted]']
df = df[df['author']!='AutoModerator']

In [44]:
# Too Slow. Don't use.

# class CustomVectorizer(CountVectorizer): 
    
#     # overwrite the build_analyzer method, allowing one to
#     # create a custom analyzer for the vectorizer
#     def build_analyzer(self):
        
#         # load stop words using CountVectorizer's built in method
#         stop_words = self.get_stop_words()
        
#         # create the analyzer that will be returned by this method
#         def analyser(doc):
            
#             # load spaCy's model for english language
#             spacy.load('en')
            
#             # instantiate a spaCy tokenizer
#             lemmatizer = spacy.lang.en.English()
            
#             # apply the preprocessing and tokenzation steps
#             doc_clean = unescape(doc).lower()
#             tokens = lemmatizer(doc_clean)
#             lemmatized_tokens = [token.lemma_ for token in tokens]
            
#             # use CountVectorizer's _word_ngrams built in method
#             # to remove stop words and extract n-grams
#             return(self._word_ngrams(lemmatized_tokens, stop_words))
#         return(analyser)

In [47]:
# create a spaCy tokenizer
spacy.load('en')
lemmatizer = spacy.lang.en.English()

# remove html entities from docs and
# set everything to lowercase
def my_preprocessor(doc):
    return(unescape(doc).lower())

# tokenize the doc and lemmatize its tokens
def my_tokenizer(doc):
    tokens = lemmatizer(doc)
    return([token.lemma_ for token in tokens])

In [58]:
# This did not give me a good topic modeling result
# custom_vec = CountVectorizer(tokenizer=my_tokenizer,
#                              stop_words='english')
# cwm = custom_vec.fit_transform(df['body'])

//anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['3', 'far', 'good', 'little', 'make', '\ufeff1'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [97]:
token_pattern_no_number=u'(?ui)\\b\\w*[a-zA-Z]+\\w*\\b'
vectorizer = CountVectorizer(stop_words = 'english', 
                             token_pattern=token_pattern_no_number, 
                             max_df=0.5,
                             min_df=2)
v_model = vectorizer.fit_transform(df['body'])

# Topic Modeling

In [8]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

### NMF

In [128]:
nmf_model = NMF(5)
topic_nmf = nmf_model.fit_transform(c_word)

In [129]:
pd.Series(topic_nmf.argmax(axis=1)).value_counts(normalize=True)

0    0.54834
1    0.20266
2    0.08459
3    0.08227
4    0.08214
dtype: float64

In [137]:
display_topics(nmf_model, vectorizer_corex.get_feature_names(), 30)


Topic  0
play, even, one, people, think, good, would, really, also, time, much, still, make, know, well, way, players, lot, see, games, better, lane, actually, playing, could, top, pretty, say, want, since

Topic  1
g2, tl, team, ig, skt, na, best, better, teams, win, vs, msi, eu, worlds, think, would, finals, beat, world, even, tournament, last, got, pvb, well, top, lost, year, still, games

Topic  2
like, feel, something, looks, seems, shit, people, look, really, feels, sounds, yeah, would, playing, things, see, said, stuff, lol, know, maybe, someone, mean, felt, always, make, champs, league, saying, etc

Topic  3
game, every, early, play, games, playing, win, late, played, lane, mid, lose, minutes, got, first, single, one, lost, bad, fun, league, hard, team, gold, shit, vs, bot, back, losing, winning

Topic  4
get, team, back, go, games, need, win, enemy, getting, want, enough, hard, kill, shit, try, time, lose, dont, gold, lane, away, help, take, banned, going, someone, way, gets,

In [12]:
df['topic'] = topic_nmf.argmax(axis=1)

In [18]:
df['topic'].value_counts(normalize=True)

0    0.755664
4    0.161139
3    0.048293
1    0.029253
2    0.005651
Name: topic, dtype: float64

In [235]:
with pd.option_context('display.max_colwidth', 500):
    print (df[['topic', 'body']].sample(1))

        topic  \
186850      0   

                                                                                                                      body  
186850  That might be it: I don’t bother with ranked, as if find everyone salty enough without fake gamepoint at stake lol  


In [13]:
df2 = pd.read_pickle('cm_19_02')
df2.drop_duplicates(subset='body', keep=False, inplace=True)
df2 = df2[df2['author']!='Ilackfocus']
df2 = df2[df2['author']!='[deleted]']
df2 = df2[df2['author']!='AutoModerator']

In [26]:
v_model2 = vectorizer.transform(df2['body'])

In [27]:
topic2_nmf = nmf_model.transform(v_model2)

In [30]:
pd.Series(topic2_nmf.argmax(axis=1)).value_counts(normalize=True)

0    0.738020
4    0.172445
3    0.048796
1    0.028051
2    0.012688
dtype: float64

### LSA

In [35]:
lsa = TruncatedSVD(4)
topic_lsa = lsa.fit_transform(X)

In [36]:
pd.Series(topic_lsa.argmax(axis=1)).value_counts(normalize=True)

0    0.879674
1    0.049115
3    0.040619
2    0.030592
dtype: float64

In [37]:
display_topics(lsa, tfidf.get_feature_names(), 30)


Topic  0
game, just, like, g2, tl, play, team, ig, don, good, think, skt, people, really, better, na, games, win, best, time, know, playing, lol, did, bad, vs, players, gt, got, lane

Topic  1
g2, tl, ig, skt, vs, pvb, beat, win, na, finals, eu, team, fw, msi, bo5, teams, better, worlds, series, groups, best, fans, lost, won, tournament, tsm, wins, final, fan, fnc

Topic  2
game, skt, g2, ig, play, early, late, games, lane, playing, fun, played, pvb, lost, minutes, win, draft, lose, mid, bot, gold, pick, champ, fw, ranked, baron, kills, single, lead, mode

Topic  3
game, na, eu, best, team, lol, tl, fans, region, https, early, teams, worlds, com, world, players, msi, late, player, c9, international, vs, win, tsm, finals, fan, won, xmithie, watch, good


# CorEx

### Anchor Lists

In [40]:
league = ['LCS', 'LEC', 'LCK', 'LPL', 'LMS','VCS', 
          'LCL', 'TCL', 'CBLOL', 'LLA', 'LJL','OPL', 'LST']
league = [item.lower() for item in league]

In [41]:
tournament = ['msi', 'world','tournament', 'region', 'na','eu']

In [42]:
hero = pd.read_csv('hero_list')
hero = hero['Hero'].values.tolist()
hero = [item.lower() for item in hero]

In [43]:
position = ['top', 'jungle','mid','adc', 'support']

In [44]:
team = ['ig','skt','tl','c9','tsm','fnc','faker','jensen','uzi','bjergsen','doublelift']

In [45]:
rank = ['ranked','rank','elo','bronze','silver','gold','master','challenger','diamond']

In [46]:
skin = ['buy','skin','skins','money']

In [47]:
anchors = [league+tournament+team, position, rank, skin, hero]

### CorEx Modeling

In [7]:
from corextopic import corextopic as ct
from corextopic import vis_topic as vt

In [8]:
import nltk
from nltk.corpus import stopwords
stopwords = stopwords.words('english')

In [11]:
infile = open('stopword_400_list','rb')
stop400 = pickle.load(infile)
infile.close()

In [13]:
stopwords= stopwords + stop400

In [15]:
token_pattern_no_number=u'(?ui)\\b\\w*[a-zA-Z]+\\w*\\b'
vectorizer_corex = CountVectorizer(stop_words = stopwords, 
                             binary=True,
                             token_pattern=token_pattern_no_number,
                             ngram_range=(1, 2),
                             max_df=0.5,
                             min_df=2,
                             max_features=40000)
c_word = vectorizer_corex.fit_transform(df['body'])
vocab = vectorizer_corex.get_feature_names()

In [16]:
c_word.shape

(476544, 40000)

In [25]:
morde = [['mordekaiser','damage'], ['morde','damage'], ['mordekaiser','ult'],['morde','ult']]

In [26]:
# seed#42 is a good one.
ct_model = ct.Corex(n_hidden=6, seed=40)
c_model_fitted = ct_model.fit(c_word, words=vocab) #, anchors=anchors, anchor_strength=4)

In [27]:
def topic_dist(fitted_model):
    topic_count = np.asarray(fitted_model.labels).sum(axis=0)
    no_topic_count = 0
    for i, topic_ngrams in enumerate(topic_count):
        print("Topic #{}: ".format(i+1), topic_count[i]/len(fitted_model.labels))
    for doc in fitted_model.labels:
        if doc.sum() == 0:
            no_topic_count += 1
    print('No Topic :', no_topic_count/len(fitted_model.labels))
topic_dist(c_model_fitted)

Topic #1:  0.11510164853612678
Topic #2:  0.16427863953800698
Topic #3:  0.11384258326618318
Topic #4:  0.1462823999462799
Topic #5:  0.12085137993553585
Topic #6:  0.059568476363148
No Topic : 0.533228830915928


In [28]:
for i, topic_ngrams in enumerate(c_model_fitted.get_topics(n_words=60)):
    topic_ngrams = [ngram[0] for ngram in topic_ngrams if ngram[1] > 0]
    print("\nTopic #{}: {}".format(i+1, ", ".join(topic_ngrams)))


Topic #1: damage, q, e, w, items, item, ap, passive, build, shield, cc, mana, kit, tank, ad, hp, kill, healing, dmg, sa, ability, kai sa, kai, health, speed, abilities, cd, heal, nerf, tanks, assassin, poke, buff, burst, mage, sustain, attack, phase, laning, wave, mages, melee, laning phase, ranged, cdr, auto, lux, mobility, aoe, target, irelia, max, bruiser, minions, scaling, cooldown, armor, akali, base, darius

Topic #2: different, elo, skill, gold, ranked, skins, lower, balance, change, fun, plat, diamond, silver, changes, agree, rank, pro, buy, single, experience, map, simply, season, climb, design, mmr, certain, comes, current, free, others, important, mechanics, skin, similar, small, overall, easier, possible, compared, mostly, general, control, obviously, argument, lp, average, whatever, potential, opinion, whether, given, ones, focus, extremely, personally, normal, lack, situation, mind

Topic #3: na, g2, eu, tl, worlds, tsm, msi, split, c9, fnc, skt, ig, region, fans, tourna